In [1]:
import os
import sys
from pathlib import Path
import time
from pyglet.window import key
import numpy as np

In [2]:
import_path = str(Path(os.getcwd()).parent.parent)
sys.path.insert(0, import_path)

from WorldModelsExperiments.breakout.model import Model, make_model, _process_frame

In [3]:
path = '/home/student/Dropbox/MA/worldmodel/worldmodel-breakout-server-version-v2/200228'
rnn_path = path + '/tf_rnn_10000/rnn.json'
vae_path = path + '/tf_vae/vae.json'
controller_path = path + '/log/breakout.cma.16.64.best.json'

render_mode = True
use_model = True

In [4]:
model = make_model(rnn_path=rnn_path, vae_path=vae_path)
model.load_model(controller_path)

INFO:tensorflow:Model using cpu.
loading file /home/student/Dropbox/MA/worldmodel/worldmodel-breakout-server-version-v2/200228/log/breakout.cma.16.64.best.json


In [5]:
reward_list = []
obs_sequence = np.zeros(shape=(1000,210,160,3), dtype=np.uint8)

#produktionsregelstrategien über unternehmensgrenzen hinweg
                        
global human_sets_pause, key_to_action
human_sets_pause = False
num_episode = 1

In [6]:
def key_press(symbol, mod):
    global human_sets_pause
    if symbol == key.SPACE:
        human_sets_pause = not human_sets_pause

#### Start normal game until pause manually

In [7]:
for episode in range(num_episode):
    total_reward = 0
    obs = model.env.reset()
    done = False
    seq_counter = 0
    
    while not done:
        model.env.render('human')
        model.env.unwrapped.viewer.window.on_key_press = key_press
        
        obs = _process_frame(obs)
        z, mu, logvar = model.encode_obs(obs)
        _, action = model.get_action(z)
        obs, reward, done, info = model.env.step(action)
         
        obs_sequence[seq_counter,:,:,:] = obs
        total_reward += reward
        seq_counter += 1
        
        if human_sets_pause:
            time.sleep(1)
            print('render for several steps done, shift with current reward: ', total_reward)
            time.sleep(2)
            human_sets_pause= False
            model.env.viewer.close()
            model.env.viewer = None
            print('close env')
            break
            
        
    if done:
        print('game episode ', str(episode), ' is done with total reward: ', total_reward)
        if render_mode:
            model.env.viewer.close()
            model.env.viewer = None
            #model.env.close()
            print('close env')
    time.sleep(2)

render for several steps done, shift with current reward:  2.0
close env


#### Video Sequence

In [8]:
import base64
import imageio
import IPython
from PIL import Image

In [9]:
def embed_mp4(filename,video):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

In [10]:
filename = 'obs_sequence.mp4'
with imageio.get_writer(filename, mode='I', macro_block_size=None) as video:    
    for image in range(seq_counter):
        video.append_data(obs_sequence[image])
embed_mp4(filename,video)

In [11]:
# PAUSE STATUS
pause_seq_counter = seq_counter
pause_state = model.state
pause_obs = obs
pause_sequence=obs_sequence[:pause_seq_counter,:,:,:]
pause_reward = total_reward
pause_gamestatus = model.env.clone_full_state()

In [12]:
print('Pause reward', pause_reward)
pause_sequence.shape

Pause reward 2.0


(445, 210, 160, 3)

#### OPTION 1 : Continue game NORMAL

In [13]:
#OPTION 1: NORMAL
# todo add several black images to simulate pause
obs_normal = np.zeros(shape=(3000,210,160,3), dtype=np.uint8)
obs_normal[:pause_seq_counter]= pause_sequence
seq_counter = pause_seq_counter +4
total_reward = pause_reward
obs = pause_obs
model.state = pause_state
model.env.restore_full_state(pause_gamestatus)
done = False

while not done:
    obs = _process_frame(obs)
    z, mu, logvar = model.encode_obs(obs)
    _, action = model.get_action(z)
    obs, reward, done, info = model.env.step(action)
    model.env.render('human')
    
    obs_normal[seq_counter,:,:,:] = obs
    total_reward += reward
    seq_counter +=1

print('Episode is done with total reward: ', total_reward)
model.env.viewer.close()
model.env.viewer = None

Episode is done with total reward:  2.0


In [14]:
filename = 'obs_sequence_normal.mp4'
with imageio.get_writer(filename, mode='I', macro_block_size=None) as video:    
    for image in range(seq_counter):
        video.append_data(obs_normal[image])
embed_mp4(filename,video)

#### OPTION 2 : Continue game SHIFT TO RIGHT

In [19]:
#OPTION 2: RIGHT
obs_right = np.zeros(shape=(3000,210,160,3), dtype=np.uint8)
obs_right[:pause_seq_counter]= pause_sequence
seq_counter = pause_seq_counter +4
total_reward = pause_reward
model.state = pause_state
model.env.restore_full_state(pause_gamestatus)
done = False

for i in range(20):
    obs, _, _, _ = model.env.step(2)
#todo state auch updaten ? 

obs_right[seq_counter,:,:,:]=obs
seq_counter +=1

while not done:
    obs = _process_frame(obs)
    z, mu, logvar = model.encode_obs(obs)
    _, action = model.get_action(z)
    NORMALobs, reward, done, info = model.env.step(action)
    model.env.render('human')
    
    obs_right[seq_counter,:,:,:] = obs
    total_reward += reward
    seq_counter +=1

print('Episode is done with total reward: ', total_reward)
model.env.viewer.close()
model.env.viewer = None

Episode is done with total reward:  2.0


In [20]:
filename = 'obs_sequence_right.mp4'
with imageio.get_writer(filename, mode='I', macro_block_size=None) as video:    
    for image in range(seq_counter):
        video.append_data(obs_right[image])
embed_mp4(filename,video) 

#### OPTION 3 : Continue game SHIFT TO LEFT

In [21]:
#OPTION 3: LEFT
obs_left = np.zeros(shape=(3000,210,160,3), dtype=np.uint8)
obs_left[:pause_seq_counter]= pause_sequence
seq_counter = pause_seq_counter +4
total_reward = pause_reward
model.state = pause_state
model.env.restore_full_state(pause_gamestatus)
done = False

for i in range(20):
    obs, _, _, _ = model.env.step(3)
#todo state auch updaten ? 

obs_left[seq_counter,:,:,:]=obs
seq_counter +=1

while not done:
    obs = _process_frame(obs)
    z, mu, logvar = model.encode_obs(obs)
    _, action = model.get_action(z)
    obs, reward, done, info = model.env.step(action)
    model.env.render('human')
    
    obs_left[seq_counter,:,:,:] = obs
    total_reward += reward
    seq_counter +=1

print('Episode is done with total reward: ', total_reward)
model.env.viewer.close()
model.env.viewer = None

Episode is done with total reward:  2.0


In [22]:
filename = 'obs_sequence_left.mp4'
with imageio.get_writer(filename, mode='I', macro_block_size=None) as video:    
    for image in range(seq_counter):
        video.append_data(obs_left[image])
embed_mp4(filename,video) 

In [23]:
obs_normal.shape

(3000, 210, 160, 3)

In [24]:
obs_left.shape

(3000, 210, 160, 3)

In [25]:
obs_right.shape

(3000, 210, 160, 3)

In [40]:
all_images = np.concatenate((obs_left, obs_normal, obs_right), axis=2)
all_images.shape

(3000, 210, 480, 3)

In [38]:
img = Image.fromarray(all_images_2[0])
img.show()

In [41]:
filename = 'obs_sequence_all.mp4'
with imageio.get_writer(filename, mode='I', macro_block_size=None) as video:    
    for image in range(len(all_images)):
        video.append_data(all_images[image])
embed_mp4(filename,video) 